In [ ]:
#| default_exp core

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Core
> NeuralForecast contains two main components, PyTorch implementations deep learning predictive models, as well as parallelization and distributed computation utilities. The first component comprises low-level PyTorch model estimator classes like `models.NBEATS` and `models.RNN`. The second component is a high-level `core.NeuralForecast` wrapper class that operates with sets of time series data stored in pandas DataFrames.

In [ ]:
#| hide
import shutil
from fastcore.test import test_eq, test_fail
from nbdev.showdoc import show_doc
from neuralforecast.utils import generate_series

In [ ]:
#| export
import os
import pickle
import warnings
from copy import deepcopy
from itertools import chain
from typing import Any, Dict, List, Optional, Union

import numpy as np
import pandas as pd
import torch
import utilsforecast.processing as ufp
from utilsforecast.compat import DataFrame, Series, pl_DataFrame, pl_Series
from utilsforecast.grouped_array import GroupedArray
from utilsforecast.target_transforms import (
    BaseTargetTransform,
    LocalBoxCox,
    LocalMinMaxScaler,            
    LocalRobustScaler,            
    LocalStandardScaler,
)
from utilsforecast.validation import validate_freq

from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.models import (
    GRU, LSTM, RNN, TCN, DeepAR, DilatedRNN,
    MLP, NHITS, NBEATS, NBEATSx,
    TFT, VanillaTransformer,
    Informer, Autoformer, FEDformer,
    StemGNN, PatchTST, TimesNet
)

In [ ]:
#| exporti
def _insample_times(
    times: np.ndarray,
    uids: Series,
    indptr: np.ndarray,
    h: int,
    freq: Union[int, str, pd.offsets.BaseOffset],
    step_size: int = 1,
    id_col: str = 'unique_id',
    time_col: str = 'ds',
) -> DataFrame:
    sizes = np.diff(indptr)
    if (sizes < h).any():
        raise ValueError('`sizes` should be greater or equal to `h`.')
    # TODO: we can just truncate here instead of raising an error
    ns, resids = np.divmod(sizes - h, step_size)
    if (resids != 0).any():
        raise ValueError('`sizes - h` should be multiples of `step_size`')
    windows_per_serie = ns + 1
    # determine the offsets for the cutoffs, e.g. 2 means the 3rd training date is a cutoff
    cutoffs_offsets = step_size * np.hstack([np.arange(w) for w in windows_per_serie])
    # start index of each serie, e.g. [0, 17] means the the second serie starts on the 18th entry
    # we repeat each of these as many times as we have windows, e.g. windows_per_serie = [2, 3]
    # would yield [0, 0, 17, 17, 17]
    start_idxs = np.repeat(indptr[:-1], windows_per_serie)
    # determine the actual indices of the cutoffs, we repeat the cutoff for the complete horizon
    # e.g. if we have two series and h=2 this could be [0, 0, 1, 1, 17, 17, 18, 18]
    # which would have the first two training dates from each serie as the cutoffs
    cutoff_idxs = np.repeat(start_idxs + cutoffs_offsets, h)
    cutoffs = times[cutoff_idxs]
    total_windows = windows_per_serie.sum()
    # determine the offsets for the actual dates. this is going to be [0, ..., h] repeated
    ds_offsets = np.tile(np.arange(h), total_windows)
    # determine the actual indices of the times
    # e.g. if we have two series and h=2 this could be [0, 1, 1, 2, 17, 18, 18, 19]
    ds_idxs = cutoff_idxs + ds_offsets
    ds = times[ds_idxs]
    if isinstance(uids, pl_Series):
        df_constructor = pl_DataFrame
    else:
        df_constructor = pd.DataFrame
    out = df_constructor(
        {
            id_col: ufp.repeat(uids, h * windows_per_serie),
            time_col: ds,
            'cutoff': cutoffs,
        }
    )
    # the first cutoff is before the first train date
    actual_cutoffs = ufp.offset_times(out['cutoff'], freq, -1)
    out = ufp.assign_columns(out, 'cutoff', actual_cutoffs)
    return out

In [ ]:
#| hide
uids = pd.Series(['id_0', 'id_1'])
indptr = np.array([0, 4, 10], dtype=np.int32)
h = 2
for step_size, freq, days in zip([1, 2], ['D', 'W-THU'], [1, 14]):
    times = np.hstack([
        pd.date_range('2000-01-01', freq=freq, periods=4),
        pd.date_range('2000-10-10', freq=freq, periods=10),
    ])    
    times_df = _insample_times(times, uids, indptr, h, freq, step_size=step_size)
    pd.testing.assert_frame_equal(
        times_df.groupby('unique_id')['ds'].min().reset_index(),
        pd.DataFrame({
            'unique_id': uids,
            'ds': times[indptr[:-1]],
        })
    )
    pd.testing.assert_frame_equal(
        times_df.groupby('unique_id')['ds'].max().reset_index(),
        pd.DataFrame({
            'unique_id': uids,
            'ds': times[indptr[1:] - 1],
        })
    )
    cutoff_deltas = (
        times_df
        .drop_duplicates(['unique_id', 'cutoff'])
        .groupby('unique_id')
        ['cutoff']
        .diff()
        .dropna()
    )
    assert cutoff_deltas.nunique() == 1
    assert cutoff_deltas.unique()[0] == pd.Timedelta(f'{days}D')

In [ ]:
#| exporti
MODEL_FILENAME_DICT = {'gru': GRU, 'lstm': LSTM, 'rnn': RNN, 
                       'tcn': TCN, 'deepar': DeepAR, 'dilatedrnn': DilatedRNN,
                       'mlp': MLP, 'nbeats': NBEATS, 'nbeatsx': NBEATSx, 'nhits': NHITS,
                       'tft': TFT,
                       'vanillatransformer': VanillaTransformer, 'informer': Informer, 'autoformer': Autoformer, 'patchtst': PatchTST,
                       'stemgnn': StemGNN,
                       'autogru': GRU, 'autolstm': LSTM, 'autornn': RNN,
                       'autotcn': TCN, 'autodeepar': DeepAR, 'autodilatedrnn': DilatedRNN,
                       'automlp': MLP, 'autonbeats': NBEATS, 'autonbeatsx': NBEATSx, 'autonhits': NHITS,
                       'autotft': TFT,
                       'autovanillatransformer': VanillaTransformer,'autoinformer': Informer, 'autoautoformer': Autoformer, 'autopatchtst': PatchTST,
                       'autofedformer': FEDformer,
                       'autostemgnn': StemGNN,
                       'autotimesnet': TimesNet,
                       }

In [ ]:
#| exporti
_type2scaler = {
    'standard': LocalStandardScaler,
    'robust': lambda: LocalRobustScaler(scale='mad'),
    'robust-iqr': lambda: LocalRobustScaler(scale='iqr'),
    'minmax': LocalMinMaxScaler,
    'boxcox': LocalBoxCox,
}

In [ ]:
#| exporti
def _id_as_idx() -> bool:
    return not bool(os.getenv("NIXTLA_ID_AS_COL", ""))

def _warn_id_as_idx():
    warnings.warn(
        "In a future version the predictions will have the id as a column. "
        "You can set the `NIXTLA_ID_AS_COL` environment variable "
        "to adopt the new behavior and to suppress this warning.",
        category=DeprecationWarning,
    )

In [ ]:
#| export
class NeuralForecast:
    
    def __init__(self, 
                 models: List[Any],
                 freq: Union[str, int],
                 local_scaler_type: Optional[str] = None):
        """
        The `core.StatsForecast` class allows you to efficiently fit multiple `NeuralForecast` models 
        for large sets of time series. It operates with pandas DataFrame `df` that identifies series 
        and datestamps with the `unique_id` and `ds` columns. The `y` column denotes the target 
        time series variable.

        Parameters
        ----------
        models : List[typing.Any]
            Instantiated `neuralforecast.models` 
            see [collection here](https://nixtla.github.io/neuralforecast/models.html).
        freq : str or int
            Frequency of the data. Must be a valid pandas or polars offset alias, or an integer.
        local_scaler_type : str, optional (default=None)
            Scaler to apply per-serie to all features before fitting, which is inverted after predicting.
            Can be 'standard', 'robust', 'robust-iqr', 'minmax' or 'boxcox'
        
        Returns
        -------
        self : NeuralForecast
            Returns instantiated `NeuralForecast` class.
        """
        assert all(model.h == models[0].h for model in models), 'All models should have the same horizon'

        self.h = models[0].h
        self.models_init = models
        self.models = [deepcopy(model) for model in self.models_init]
        self.freq = freq
        if local_scaler_type is not None and local_scaler_type not in _type2scaler:
            raise ValueError(f'scaler_type must be one of {_type2scaler.keys()}')
        self.local_scaler_type = local_scaler_type
        self.scalers_: Dict[str, BaseTargetTransform]

        # Flags and attributes
        self._fitted = False

    def _scalers_fit_transform(self, dataset: TimeSeriesDataset) -> None:
        self.scalers_ = {}        
        if self.local_scaler_type is None:
            return None
        for i, col in enumerate(dataset.temporal_cols):
            if col == 'available_mask':
                continue
            self.scalers_[col] = _type2scaler[self.local_scaler_type]()                
            ga = GroupedArray(dataset.temporal[:, i].numpy(), dataset.indptr)
            dataset.temporal[:, i] = torch.from_numpy(self.scalers_[col].fit_transform(ga))
    
    def _scalers_transform(self, dataset: TimeSeriesDataset) -> None:
        if not self.scalers_:
            return None
        for i, col in enumerate(dataset.temporal_cols):
            scaler = self.scalers_.get(col, None)
            if scaler is None:
                continue
            ga = GroupedArray(dataset.temporal[:, i].numpy(), dataset.indptr)
            dataset.temporal[:, i] = torch.from_numpy(scaler.transform(ga))

    def _scalers_target_inverse_transform(self, data: np.ndarray, indptr: np.ndarray) -> np.ndarray:
        if not self.scalers_:
            return data
        for i in range(data.shape[1]):
            ga = GroupedArray(data[:, i], indptr)
            data[:, i] = self.scalers_[self.target_col].inverse_transform(ga)
        return data

    def _prepare_fit(self, df, static_df, sort_df, predict_only, id_col, time_col, target_col):
        #TODO: uids, last_dates and ds should be properties of the dataset class. See github issue.
        self.id_col = id_col
        self.time_col = time_col
        self.target_col = target_col
        dataset, uids, last_dates, ds = TimeSeriesDataset.from_df(
            df=df,
            static_df=static_df,
            sort_df=sort_df,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
        )
        if predict_only:
            self._scalers_transform(dataset)
        else:
            self._scalers_fit_transform(dataset)
        return dataset, uids, last_dates, ds

    def fit(self,
        df: Optional[DataFrame] = None,
        static_df: Optional[DataFrame] = None,
        val_size: Optional[int] = 0,
        sort_df: bool = True,
        use_init_models: bool = False,
        verbose: bool = False,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
    ) -> None:
        """Fit the core.NeuralForecast.

        Fit `models` to a large set of time series from DataFrame `df`.
        and store fitted models for later inspection.

        Parameters
        ----------
        df : pandas or polars DataFrame, optional (default=None)
            DataFrame with columns [`unique_id`, `ds`, `y`] and exogenous variables.
            If None, a previously stored dataset is required.
        static_df : pandas or polars DataFrame, optional (default=None)
            DataFrame with columns [`unique_id`] and static exogenous.
        val_size : int, optional (default=0)
            Size of validation set.
        sort_df : bool, optional (default=False)
            Sort `df` before fitting.
        use_init_models : bool, optional (default=False)
            Use initial model passed when NeuralForecast object was instantiated.
        verbose : bool (default=False)
            Print processing steps.
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.

        Returns
        -------
        self : NeuralForecast
            Returns `NeuralForecast` class with fitted `models`.
        """
        if (df is None) and not (hasattr(self, 'dataset')):
            raise Exception('You must pass a DataFrame or have one stored.')

        # Model and datasets interactions protections
        if (any(model.early_stop_patience_steps>0 for model in self.models)) \
            and (val_size==0):
                raise Exception('Set val_size>0 if early stopping is enabled.')

        # Process and save new dataset (in self)
        if df is not None:
            validate_freq(df[time_col], self.freq)
            self.dataset, self.uids, self.last_dates, self.ds = self._prepare_fit(
                df=df,
                static_df=static_df,
                sort_df=sort_df,
                predict_only=False,
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
            )
            self.sort_df = sort_df
        else:
            if verbose: print('Using stored dataset.')

        if val_size is not None:
            if self.dataset.min_size < val_size:
                warnings.warn('Validation set size is larger than the shorter time-series.')

        # Recover initial model if use_init_models
        if use_init_models:
            self.models = [deepcopy(model) for model in self.models_init]
            if self._fitted:
                print('WARNING: Deleting previously fitted models.')

        for model in self.models:
            model.fit(self.dataset, val_size=val_size)

        self._fitted = True

    def make_future_dataframe(self, df: Optional[DataFrame] = None) -> DataFrame:
        """Create a dataframe with all ids and future times in the forecasting horizon.

        Parameters
        ----------
        df : pandas or polars DataFrame, optional (default=None)
            DataFrame with columns [`unique_id`, `ds`, `y`] and exogenous variables.
            Only required if this is different than the one used in the fit step.
        """
        if not self._fitted:
            raise Exception('You must fit the model first.')
        if df is not None:
            df = ufp.sort(df, by=[self.id_col, self.time_col])
            last_times_by_id = ufp.group_by_agg(
                df,
                by=self.id_col,
                aggs={self.time_col: 'max'},
                maintain_order=True,
            )
            uids = last_times_by_id[self.id_col]
            last_times = last_times_by_id[self.time_col]
        else:
            uids = self.uids
            last_times = self.last_dates
        return ufp.make_future_dataframe(
            uids=uids,
            last_times=last_times,
            freq=self.freq,
            h=self.h,
            id_col=self.id_col,
            time_col=self.time_col,
        )

    def get_missing_future(
        self, futr_df: DataFrame, df: Optional[DataFrame] = None
    ) -> DataFrame:
        """Get the missing ids and times combinations in `futr_df`.
        
        Parameters
        ----------
        futr_df : pandas or polars DataFrame
            DataFrame with [`unique_id`, `ds`] columns and `df`'s future exogenous.
        df : pandas or polars DataFrame, optional (default=None)
            DataFrame with columns [`unique_id`, `ds`, `y`] and exogenous variables.
            Only required if this is different than the one used in the fit step.
        """
        expected = self.make_future_dataframe(df)
        ids = [self.id_col, self.time_col]
        return ufp.anti_join(expected, futr_df[ids], on=ids)

    def predict(self,
                df: Optional[DataFrame] = None,
                static_df: Optional[DataFrame] = None,
                futr_df: Optional[DataFrame] = None,
                sort_df: bool = True,
                verbose: bool = False,
                **data_kwargs):
        """Predict with core.NeuralForecast.

        Use stored fitted `models` to predict large set of time series from DataFrame `df`.        

        Parameters
        ----------
        df : pandas or polars DataFrame, optional (default=None)
            DataFrame with columns [`unique_id`, `ds`, `y`] and exogenous variables.
            If a DataFrame is passed, it is used to generate forecasts.
        static_df : pandas or polars DataFrame, optional (default=None)
            DataFrame with columns [`unique_id`] and static exogenous.
        futr_df : pandas or polars DataFrame, optional (default=None)
            DataFrame with [`unique_id`, `ds`] columns and `df`'s future exogenous.
        sort_df : bool (default=True)
            Sort `df` before fitting.
        verbose : bool (default=False)
            Print processing steps.
        data_kwargs : kwargs
            Extra arguments to be passed to the dataset within each model.

        Returns
        -------
        fcsts_df : pandas or polars DataFrame
            DataFrame with insample `models` columns for point predictions and probabilistic
            predictions for all fitted `models`.    
        """
        if (df is None) and not (hasattr(self, 'dataset')):
            raise Exception('You must pass a DataFrame or have one stored.')

        if not self._fitted:
            raise Exception("You must fit the model before predicting.")

        needed_futr_exog = set(chain.from_iterable(getattr(m, 'futr_exog_list', []) for m in self.models))
        if needed_futr_exog:
            if futr_df is None:
                raise ValueError(
                    f'Models require the following future exogenous features: {needed_futr_exog}. '
                    'Please provide them through the `futr_df` argument.'
                )
            else:
                missing = needed_futr_exog - set(futr_df.columns)
                if missing:
                    raise ValueError(f'The following features are missing from `futr_df`: {missing}')

        # Process new dataset but does not store it.
        if df is not None:
            validate_freq(df[self.time_col], self.freq)
            dataset, uids, last_dates, _ = self._prepare_fit(
                df=df,
                static_df=static_df,
                sort_df=sort_df,
                predict_only=True,
                id_col=self.id_col,
                time_col=self.time_col,
                target_col=self.target_col,
            )
        else:
            dataset = self.dataset
            uids = self.uids
            last_dates = self.last_dates
            if verbose: print('Using stored dataset.')

        cols = []
        count_names = {'model': 0}
        for model in self.models:
            model_name = repr(model)
            count_names[model_name] = count_names.get(model_name, -1) + 1
            if count_names[model_name] > 0:
                model_name += str(count_names[model_name])
            cols += [model_name + n for n in model.loss.output_names]

        # Placeholder dataframe for predictions with unique_id and ds
        fcsts_df = ufp.make_future_dataframe(
            uids=uids,
            last_times=last_dates,
            freq=self.freq,
            h=self.h,
            id_col=self.id_col,
            time_col=self.time_col,
        )

        # Update and define new forecasting dataset
        if futr_df is None:
            futr_df = fcsts_df
        else:
            futr_orig_rows = futr_df.shape[0]
            futr_df = ufp.join(futr_df, fcsts_df, on=[self.id_col, self.time_col])
            if futr_df.shape[0] < fcsts_df.shape[0]:
                if df is None:
                    expected_cmd = 'make_future_dataframe()'
                    missing_cmd = 'get_missing_future(futr_df)'
                else:
                    expected_cmd = 'make_future_dataframe(df)'
                    missing_cmd = 'get_missing_future(futr_df, df)'
                raise ValueError(
                    'There are missing combinations of ids and times in `futr_df`.\n'
                    f'You can run the `{expected_cmd}` method to get the expected combinations or '
                    f'the `{missing_cmd}` method to get the missing combinations.'
                )
            if futr_orig_rows > futr_df.shape[0]:
                dropped_rows = futr_orig_rows - futr_df.shape[0]
                warnings.warn(
                    f'Dropped {dropped_rows:,} unused rows from `futr_df`.'
                )
            if any(ufp.is_none(futr_df[col]).any() for col in needed_futr_exog):
                raise ValueError('Found null values in `futr_df`')
        futr_dataset = dataset.align(
            futr_df,
            id_col=self.id_col,
            time_col=self.time_col,
            target_col=self.target_col,
        )
        self._scalers_transform(futr_dataset)
        dataset = dataset.append(futr_dataset)

        col_idx = 0
        fcsts = np.full((self.h * len(uids), len(cols)), fill_value=np.nan)
        for model in self.models:
            old_test_size = model.get_test_size()
            model.set_test_size(self.h) # To predict h steps ahead
            model_fcsts = model.predict(dataset=dataset, **data_kwargs)
            # Append predictions in memory placeholder
            output_length = len(model.loss.output_names)
            fcsts[:, col_idx : col_idx + output_length] = model_fcsts
            col_idx += output_length
            model.set_test_size(old_test_size) # Set back to original value
        if self.scalers_:
            indptr = np.append(0, np.full(len(uids), self.h).cumsum())
            fcsts = self._scalers_target_inverse_transform(fcsts, indptr)

        # Declare predictions pd.DataFrame
        if isinstance(self.uids, pl_Series):
            fcsts = pl_DataFrame(dict(zip(cols, fcsts.T)))
        else:
            fcsts = pd.DataFrame(fcsts, columns=cols)
        fcsts_df = ufp.horizontal_concat([fcsts_df, fcsts])
        if isinstance(fcsts_df, pd.DataFrame) and _id_as_idx():
            _warn_id_as_idx()
            fcsts_df = fcsts_df.set_index(self.id_col)
        return fcsts_df
    
    def cross_validation(self,
                         df: Optional[pd.DataFrame] = None,
                         static_df: Optional[pd.DataFrame] = None,
                         n_windows: int = 1,
                         step_size: int = 1,
                         val_size: Optional[int] = 0, 
                         test_size: Optional[int] = None,
                         sort_df: bool = True,
                         use_init_models: bool = False,
                         verbose: bool = False,
                         id_col: str = 'unique_id',
                         time_col: str = 'ds',
                         target_col: str = 'y',
                         **data_kwargs):
        """Temporal Cross-Validation with core.NeuralForecast.

        `core.NeuralForecast`'s cross-validation efficiently fits a list of NeuralForecast 
        models through multiple windows, in either chained or rolled manner.

        Parameters
        ----------
        df : pandas.DataFrame, optional (default=None)
            DataFrame with columns [`unique_id`, `ds`, `y`] and exogenous variables.
            If None, a previously stored dataset is required.
        static_df : pandas.DataFrame, optional (default=None)
            DataFrame with columns [`unique_id`] and static exogenous.
        n_windows : int (default=1)
            Number of windows used for cross validation.
        step_size : int (default=1)
            Step size between each window.
        val_size : int, optional (default=None)
            Length of validation size. If passed, set `n_windows=None`.
        test_size : int, optional (default=None)
            Length of test size. If passed, set `n_windows=None`.
        sort_df : bool (default=True)
            Sort `df` before fitting.
        use_init_models : bool, option (default=False)
            Use initial model passed when object was instantiated.
        verbose : bool (default=False)
            Print processing steps.
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.            
        data_kwargs : kwargs
            Extra arguments to be passed to the dataset within each model.

        Returns
        -------
        fcsts_df : pandas.DataFrame
            DataFrame with insample `models` columns for point predictions and probabilistic
            predictions for all fitted `models`.    
        """
        if (df is None) and not (hasattr(self, 'dataset')):
            raise Exception('You must pass a DataFrame or have one stored.')

        # Process and save new dataset (in self)
        if df is not None:
            validate_freq(df[time_col], self.freq)
            self.dataset, self.uids, self.last_dates, self.ds = self._prepare_fit(
                df=df,
                static_df=static_df,
                sort_df=sort_df,
                predict_only=False,
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
            )
            self.sort_df = sort_df
        else:
            if verbose: print('Using stored dataset.')

        # Recover initial model if use_init_models.
        if use_init_models:
            self.models = [deepcopy(model) for model in self.models_init]
            if self._fitted:
                print('WARNING: Deleting previously fitted models.')

        cols = []
        count_names = {'model': 0}
        for model in self.models:
            model_name = repr(model)
            count_names[model_name] = count_names.get(model_name, -1) + 1
            if count_names[model_name] > 0:
                model_name += str(count_names[model_name])
            cols += [model_name + n for n in model.loss.output_names]            

        h = self.models[0].h
        if test_size is None:
            test_size = h + step_size * (n_windows - 1)
        elif n_windows is None:
            if (test_size - h) % step_size:
                raise Exception('`test_size - h` should be module `step_size`')
            n_windows = int((test_size - h) / step_size) + 1
        elif (n_windows is None) and (test_size is None):
            raise Exception('you must define `n_windows` or `test_size`')
        else:
            raise Exception('you must define `n_windows` or `test_size` but not both')

        if val_size is not None:
            if self.dataset.min_size < (val_size+test_size):
                warnings.warn('Validation and test sets are larger than the shorter time-series.')

        fcsts_df = ufp.cv_times(
            times=self.ds,
            uids=self.uids,
            indptr=self.dataset.indptr,
            h=self.h,
            test_size=test_size,
            step_size=step_size,
            id_col=id_col,
            time_col=time_col,
        )
        # the cv_times is sorted by window and then id
        fcsts_df = ufp.sort(fcsts_df, [id_col, 'cutoff', time_col])

        col_idx = 0
        fcsts = np.full((self.dataset.n_groups * h * n_windows, len(cols)),
                         np.nan, dtype=np.float32)
        
        for model in self.models:
            model.fit(dataset=self.dataset,
                        val_size=val_size, 
                        test_size=test_size)
            model_fcsts = model.predict(self.dataset, step_size=step_size, **data_kwargs)

            # Append predictions in memory placeholder
            output_length = len(model.loss.output_names)
            fcsts[:,col_idx:(col_idx + output_length)] = model_fcsts
            col_idx += output_length
        if self.scalers_:            
            indptr = np.append(0, np.full(self.dataset.n_groups, self.h * n_windows).cumsum())
            fcsts = self._scalers_target_inverse_transform(fcsts, indptr)

        self._fitted = True

        # Add predictions to forecasts DataFrame
        if isinstance(self.uids, pl_Series):
            fcsts = pl_DataFrame(dict(zip(cols, fcsts.T)))
        else:
            fcsts = pd.DataFrame(fcsts, columns=cols)
        fcsts_df = ufp.horizontal_concat([fcsts_df, fcsts])

        # Add original input df's y to forecasts DataFrame    
        fcsts_df = ufp.join(fcsts_df, df, how='left', on=[id_col, time_col])
        if isinstance(fcsts_df, pd.DataFrame) and _id_as_idx():
            _warn_id_as_idx()
            fcsts_df = fcsts_df.set_index(id_col)
        return fcsts_df

    def predict_insample(self, step_size: int = 1):
        """Predict insample with core.NeuralForecast.

        `core.NeuralForecast`'s `predict_insample` uses stored fitted `models`
        to predict historic values of a time series from the stored dataframe.

        Parameters
        ----------
        step_size : int (default=1)
            Step size between each window.

        Returns
        -------
        fcsts_df : pandas.DataFrame
            DataFrame with insample predictions for all fitted `models`.    
        """
        if not self._fitted:
            raise Exception('The models must be fitted first with `fit` or `cross_validation`.')

        for model in self.models:
            if model.SAMPLING_TYPE == 'recurrent':
                warnings.warn(f'Predict insample might not provide accurate predictions for \
                       recurrent model {repr(model)} class yet due to scaling.')
                print(f'WARNING: Predict insample might not provide accurate predictions for \
                      recurrent model {repr(model)} class yet due to scaling.')
        
        cols = []
        count_names = {'model': 0}
        for model in self.models:
            model_name = repr(model)
            count_names[model_name] = count_names.get(model_name, -1) + 1
            if count_names[model_name] > 0:
                model_name += str(count_names[model_name])
            cols += [model_name + n for n in model.loss.output_names]

        # Remove test set from dataset and last dates
        test_size = self.models[0].get_test_size()
        if test_size>0:
            trimmed_dataset = TimeSeriesDataset.trim_dataset(dataset=self.dataset,
                                                     right_trim=test_size,
                                                     left_trim=0)
            new_idxs = np.hstack(
                [
                    np.arange(self.dataset.indptr[i], self.dataset.indptr[i + 1] - test_size)
                    for i in range(self.dataset.n_groups)
                ]
            )
            times = self.ds[new_idxs]
        else:
            trimmed_dataset = self.dataset
            times = self.ds

        # Generate dates
        fcsts_df = _insample_times(
            times=times,
            uids=self.uids,
            indptr=trimmed_dataset.indptr,
            h=self.h,
            freq=self.freq,
            step_size=step_size,
            id_col=self.id_col,
            time_col=self.time_col,
        )

        col_idx = 0
        fcsts = np.full((len(fcsts_df), len(cols)), np.nan, dtype=np.float32)

        for model in self.models:
            # Test size is the number of periods to forecast (full size of trimmed dataset)
            model.set_test_size(test_size=trimmed_dataset.max_size)

            # Predict
            model_fcsts = model.predict(trimmed_dataset, step_size=step_size)
            # Append predictions in memory placeholder
            output_length = len(model.loss.output_names)
            fcsts[:,col_idx:(col_idx + output_length)] = model_fcsts
            col_idx += output_length          
            model.set_test_size(test_size=test_size) # Set original test_size

        # original y
        original_y = {
            self.id_col: ufp.repeat(self.uids, np.diff(self.dataset.indptr)),
            self.time_col: self.ds,
            self.target_col: self.dataset.temporal[:, 0].numpy(),
        }

        # Add predictions to forecasts DataFrame
        if isinstance(self.uids, pl_Series):
            fcsts = pl_DataFrame(dict(zip(cols, fcsts.T)))
            Y_df = pl_DataFrame(original_y)
        else:
            fcsts = pd.DataFrame(fcsts, columns=cols)
            Y_df = pd.DataFrame(original_y).reset_index(drop=True)
        fcsts_df = ufp.horizontal_concat([fcsts_df, fcsts])

        # Add original input df's y to forecasts DataFrame
        fcsts_df = ufp.join(fcsts_df, Y_df, how='left', on=[self.id_col, self.time_col])
        if self.scalers_:
            sizes = ufp.counts_by_id(fcsts_df, self.id_col)['counts'].to_numpy()
            indptr = np.append(0, sizes.cumsum())
            invert_cols = cols + [self.target_col]
            fcsts_df[invert_cols] = self._scalers_target_inverse_transform(
                fcsts_df[invert_cols].to_numpy(),
                indptr
            )
        if isinstance(fcsts_df, pd.DataFrame) and _id_as_idx():
            _warn_id_as_idx()
            fcsts_df = fcsts_df.set_index(self.id_col)            
        return fcsts_df
        
    # Save list of models with pytorch lightning save_checkpoint function
    def save(self, path: str, model_index: Optional[List]=None, save_dataset: bool=True, overwrite: bool=False):
        """Save NeuralForecast core class.

        `core.NeuralForecast`'s method to save current status of models, dataset, and configuration.
        Note that by default the `models` are not saving training checkpoints to save disk memory,
        to get them change the individual model `**trainer_kwargs` to include `enable_checkpointing=True`.

        Parameters
        ----------
        path : str
            Directory to save current status.
        model_index : list, optional (default=None)
            List to specify which models from list of self.models to save.
        save_dataset : bool (default=True)
            Whether to save dataset or not.
        overwrite : bool (default=False)
            Whether to overwrite files or not.
        """
        # Standarize path without '/'
        if path[-1] == '/':
            path = path[:-1]

        # Model index list
        if model_index is None:
            model_index = list(range(len(self.models)))

        # Create directory if not exists
        os.makedirs(path, exist_ok = True)

        # Check if directory is empty to protect overwriting files
        dir = os.listdir(path)
        
        # Checking if the list is empty or not
        if (len(dir) > 0) and (not overwrite):
            raise Exception('Directory is not empty. Set `overwrite=True` to overwrite files.')

        # Save models
        count_names = {'model': 0}
        for i, model in enumerate(self.models):
            # Skip model if not in list
            if i not in model_index:
                continue

            model_name = repr(model).lower().replace('_', '')
            count_names[model_name] = count_names.get(model_name, -1) + 1
            model.save(f"{path}/{model_name}_{count_names[model_name]}.ckpt")

        # Save dataset
        if (save_dataset) and (hasattr(self, 'dataset')):
            with open(f"{path}/dataset.pkl", "wb") as f:
                pickle.dump(self.dataset, f)
        elif save_dataset:
            raise Exception('You need to have a stored dataset to save it, \
                             set `save_dataset=False` to skip saving dataset.')

        # Save configuration and parameters
        config_dict = {
            "h": self.h,
            "freq": self.freq,
            "uids": self.uids,
            "last_dates": self.last_dates,
            "ds": self.ds,
            "sort_df": self.sort_df,
            "_fitted": self._fitted,
            "local_scaler_type": self.local_scaler_type,
            "scalers_": self.scalers_,
            "id_col": self.id_col,
            "time_col": self.time_col,
            "target_col": self.target_col,
        }

        with open(f"{path}/configuration.pkl", "wb") as f:
                pickle.dump(config_dict, f)

    @staticmethod
    def load(path, verbose=False, **kwargs):
        """Load NeuralForecast

        `core.NeuralForecast`'s method to load checkpoint from path.

        Parameters
        -----------
        path : str
            Directory to save current status.
        kwargs
            Additional keyword arguments to be passed to the function
            `load_from_checkpoint`.

        Returns
        -------
        result : NeuralForecast
            Instantiated `NeuralForecast` class.
        """
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

        # Load models
        models_ckpt = [f for f in files if f.endswith('.ckpt')]
        if len(models_ckpt) == 0:
            raise Exception('No model found in directory.') 
        
        if verbose: print(10 * '-' + ' Loading models ' + 10 * '-')
        models = []
        for model in models_ckpt:
            model_name = model.split('_')[0]
            models.append(MODEL_FILENAME_DICT[model_name].load_from_checkpoint(f"{path}/{model}", **kwargs))
            if verbose: print(f"Model {model_name} loaded.")

        if verbose: print(10*'-' + ' Loading dataset ' + 10*'-')
        # Load dataset
        if 'dataset.pkl' in files:
            with open(f"{path}/dataset.pkl", "rb") as f:
                dataset = pickle.load(f)
            if verbose: print('Dataset loaded.')
        else:
            dataset = None
            if verbose: print('No dataset found in directory.')
        
        if verbose: print(10*'-' + ' Loading configuration ' + 10*'-')
        # Load configuration
        if 'configuration.pkl' in files:
            with open(f"{path}/configuration.pkl", "rb") as f:
                config_dict = pickle.load(f)
            if verbose: print('Configuration loaded.')
        else:
            raise Exception('No configuration found in directory.')

        # Create NeuralForecast object
        neuralforecast = NeuralForecast(
            models=models,
            freq=config_dict['freq'],
            local_scaler_type=config_dict['local_scaler_type'],
        )

        # Dataset
        if dataset is not None:
            neuralforecast.dataset = dataset
            restore_attrs = [
                'uids',
                'last_dates',
                'ds',
                'sort_df',
                'id_col',
                'time_col',
                'target_col',
            ]
            for attr in restore_attrs:
                setattr(neuralforecast, attr, config_dict[attr])

        # Fitted flag
        neuralforecast._fitted = config_dict['_fitted']

        neuralforecast.scalers_ = config_dict['scalers_']

        return neuralforecast

In [ ]:
#| hide
import logging
import warnings

In [ ]:
#| hide
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

In [ ]:
show_doc(NeuralForecast.fit, title_level=3)

In [ ]:
show_doc(NeuralForecast.predict, title_level=3)

In [ ]:
show_doc(NeuralForecast.cross_validation, title_level=3)

In [ ]:
show_doc(NeuralForecast.predict_insample, title_level=3)

In [ ]:
show_doc(NeuralForecast.save, title_level=3)

In [ ]:
show_doc(NeuralForecast.load, title_level=3)

In [ ]:
#| hide
import matplotlib.pyplot as plt
import pytorch_lightning as pl

import neuralforecast
from ray import tune

from neuralforecast.auto import (
    AutoMLP, AutoNBEATS, 
    AutoRNN, AutoTCN, AutoDilatedRNN,
)

from neuralforecast.models.rnn import RNN
from neuralforecast.models.tcn import TCN
from neuralforecast.models.deepar import DeepAR
from neuralforecast.models.dilated_rnn import DilatedRNN

from neuralforecast.models.mlp import MLP
from neuralforecast.models.nhits import NHITS
from neuralforecast.models.nbeats import NBEATS
from neuralforecast.models.nbeatsx import NBEATSx

from neuralforecast.models.tft import TFT
from neuralforecast.models.vanillatransformer import VanillaTransformer
from neuralforecast.models.informer import Informer
from neuralforecast.models.autoformer import Autoformer

from neuralforecast.models.stemgnn import StemGNN

from neuralforecast.losses.pytorch import MQLoss, MAE, MSE
from neuralforecast.utils import AirPassengersDF, AirPassengersPanel, AirPassengersStatic

In [ ]:
#| hide
AirPassengersPanel_train = AirPassengersPanel[AirPassengersPanel['ds'] < AirPassengersPanel['ds'].values[-12]].reset_index(drop=True)
AirPassengersPanel_test = AirPassengersPanel[AirPassengersPanel['ds'] >= AirPassengersPanel['ds'].values[-12]].reset_index(drop=True)
AirPassengersPanel_test['y'] = np.nan
AirPassengersPanel_test['y_[lag12]'] = np.nan

In [ ]:
#| hide
# id as index warnings
models = [
    NHITS(h=12, input_size=12, max_steps=1)
]
nf = NeuralForecast(models=models, freq='M')
nf.fit(df=AirPassengersPanel_train)
with warnings.catch_warnings(record=True) as issued_warnings:
    warnings.simplefilter('always', category=DeprecationWarning)
    nf.predict()
    nf.predict_insample()
    nf.cross_validation(df=AirPassengersPanel_train)
id_warnings = [
    w for w in issued_warnings if 'the predictions will have the id as a column' in str(w.message)
]
assert len(id_warnings) == 3

In [ ]:
os.environ['NIXTLA_ID_AS_COL'] = '1'

In [ ]:
#| hide
# Unitest for early stopping without val_size protection
models = [
    NHITS(h=12, input_size=12, max_steps=1, early_stop_patience_steps=5)
]
nf = NeuralForecast(models=models, freq='M')
test_fail(nf.fit,
          contains='Set val_size>0 if early stopping is enabled.',
          args=(AirPassengersPanel_train,))

In [ ]:
#| hide
# test fit+cross_validation behaviour
models = [NHITS(h=12, input_size=24, max_steps=10)]
nf = NeuralForecast(models=models, freq='M')
nf.fit(AirPassengersPanel_train)
init_fcst = nf.predict()
init_cv = nf.cross_validation(AirPassengersPanel_train, use_init_models=True)
after_cv = nf.cross_validation(AirPassengersPanel_train, use_init_models=True)
nf.fit(AirPassengersPanel_train, use_init_models=True)
after_fcst = nf.predict()
test_eq(init_cv, after_cv)
test_eq(init_fcst, after_fcst)

In [ ]:
#| hide
# test scaling
models = [NHITS(h=12, input_size=24, max_steps=10)]
models_exog = [NHITS(h=12, input_size=12, max_steps=10, hist_exog_list=['trend'], futr_exog_list=['trend'])]

# fit+predict
nf = NeuralForecast(models=models, freq='M', local_scaler_type='standard')
nf.fit(AirPassengersPanel_train)
scaled_fcst = nf.predict()
# check that the forecasts are similar to the one without scaling
np.testing.assert_allclose(
    init_fcst['NHITS'].values,
    scaled_fcst['NHITS'].values,
    rtol=0.3,
)
# with exog
nf = NeuralForecast(models=models_exog, freq='M', local_scaler_type='standard')
nf.fit(AirPassengersPanel_train)
scaled_exog_fcst = nf.predict(futr_df=AirPassengersPanel_test)
# check that the forecasts are similar to the one without exog
np.testing.assert_allclose(
    scaled_fcst['NHITS'].values,
    scaled_exog_fcst['NHITS'].values,
    rtol=0.3,
)

# CV
nf = NeuralForecast(models=models, freq='M', local_scaler_type='robust-iqr')
cv_res = nf.cross_validation(AirPassengersPanel)
# check that the forecasts are similar to the original values (originals are restored directly from the df)
np.testing.assert_allclose(
    cv_res['NHITS'].values,
    cv_res['y'].values,
    rtol=0.3,
)
# with exog
nf = NeuralForecast(models=models_exog, freq='M', local_scaler_type='robust-iqr')
cv_res_exog = nf.cross_validation(AirPassengersPanel)
# check that the forecasts are similar to the original values (originals are restored directly from the df)
np.testing.assert_allclose(
    cv_res_exog['NHITS'].values,
    cv_res_exog['y'].values,
    rtol=0.2,
)

# fit+predict_insample
nf = NeuralForecast(models=models, freq='M', local_scaler_type='minmax')
nf.fit(AirPassengersPanel_train)
insample_res = (
    nf.predict_insample()
    .groupby('unique_id').tail(-12) # first values aren't reliable
    .merge(
        AirPassengersPanel_train[['unique_id', 'ds', 'y']],
        on=['unique_id', 'ds'],
        how='left',
        suffixes=('_actual', '_expected'),
    )
)
# y is inverted correctly
np.testing.assert_allclose(
    insample_res['y_actual'].values,
    insample_res['y_expected'].values,
    rtol=1e-5,
)
# predictions are in the same scale
np.testing.assert_allclose(
    insample_res['NHITS'].values,
    insample_res['y_expected'].values,
    rtol=0.7,
)
# with exog
nf = NeuralForecast(models=models_exog, freq='M', local_scaler_type='minmax')
nf.fit(AirPassengersPanel_train)
insample_res_exog = (
    nf.predict_insample()
    .groupby('unique_id').tail(-12) # first values aren't reliable
    .merge(
        AirPassengersPanel_train[['unique_id', 'ds', 'y']],
        on=['unique_id', 'ds'],
        how='left',
        suffixes=('_actual', '_expected'),
    )
)
# y is inverted correctly
np.testing.assert_allclose(
    insample_res_exog['y_actual'].values,
    insample_res_exog['y_expected'].values,
    rtol=1e-5,
)
# predictions are similar than without exog
np.testing.assert_allclose(
    insample_res['NHITS'].values,
    insample_res_exog['NHITS'].values,
    rtol=0.2,
)

In [ ]:
#| hide
# test futr_df contents
models = [NHITS(h=6, input_size=24, max_steps=10, hist_exog_list=['trend'], futr_exog_list=['trend'])]
nf = NeuralForecast(models=models, freq='M')
nf.fit(AirPassengersPanel_train)
# not enough rows in futr_df raises an error
test_fail(lambda: nf.predict(futr_df=AirPassengersPanel_test.head()), contains='There are missing combinations')
# extra rows issues a warning
with warnings.catch_warnings(record=True) as issued_warnings:
    warnings.simplefilter('always', UserWarning)
    nf.predict(futr_df=AirPassengersPanel_test)
assert any('Dropped 12 unused rows' in str(w.message) for w in issued_warnings)
# models require futr_df and not provided raises an error
test_fail(lambda: nf.predict(), contains="Models require the following future exogenous features: {'trend'}") 
# missing feature in futr_df raises an error
test_fail(lambda: nf.predict(futr_df=AirPassengersPanel_test.drop(columns='trend')), contains="missing from `futr_df`: {'trend'}")
# null values in futr_df raises an error
test_fail(lambda: nf.predict(futr_df=AirPassengersPanel_test.assign(trend=np.nan)), contains='Found null values in `futr_df`')

In [ ]:
#| hide
# Test inplace model fitting
models = [MLP(h=12, input_size=12, max_steps=1, scaler_type='robust')]
initial_weights = models[0].mlp[0].weight.detach().clone()
fcst = NeuralForecast(models=models, freq='M')
fcst.fit(df=AirPassengersPanel_train, static_df=AirPassengersStatic, use_init_models=True)
after_weights = fcst.models_init[0].mlp[0].weight.detach().clone()
assert np.allclose(initial_weights, after_weights), 'init models should not be modified'
assert len(fcst.models[0].train_trajectories)>0, 'models stored trajectories should not be empty'

In [ ]:
#| hide
# Test predict_insample
test_size = 12
n_series = 2
h = 12

config = {'input_size': tune.choice([12, 24]), 
          'hidden_size': 128,
          'max_steps': 1,
          'val_check_steps': 1,
          'step_size': 12}

models = [
    NHITS(h=h, input_size=24, loss=MQLoss(level=[80]), max_steps=1, alias='NHITS', scaler_type=None),
    AutoMLP(h=12, config=config, cpus=1, num_samples=1),
    RNN(h=h, input_size=-1, loss=MAE(), max_steps=1, alias='RNN', scaler_type=None),
    ]

nf = NeuralForecast(models=models, freq='M')
cv = nf.cross_validation(df=AirPassengersPanel_train, static_df=AirPassengersStatic, val_size=0, test_size=test_size, n_windows=None)

forecasts = nf.predict_insample(step_size=1)

expected_size = n_series*((len(AirPassengersPanel_train)//n_series-test_size)-h+1)*h
assert len(forecasts) == expected_size, 'Shape mistmach in predict_insample'

In [ ]:
#| hide
# tests aliases
config_drnn = {'input_size': tune.choice([-1]), 
               'encoder_hidden_size': tune.choice([5, 10]),
               'max_steps': 1,
               'val_check_steps': 1,
               'step_size': 1}
models = [
    # test Auto
    AutoDilatedRNN(h=12, config=config_drnn, cpus=1, num_samples=2, alias='AutoDIL'),
    # test BaseWindows
    NHITS(h=12, input_size=24, loss=MQLoss(level=[80]), max_steps=1, alias='NHITSMQ'),
    # test BaseRecurrent
    RNN(h=12, input_size=-1, encoder_hidden_size=10, max_steps=1,
            stat_exog_list=['airline1'],
            futr_exog_list=['trend'], hist_exog_list=['y_[lag12]'], alias='MyRNN'),
    # test BaseMultivariate
    StemGNN(h=12, input_size=24, n_series=2, max_steps=1, scaler_type='robust', alias='StemMulti'),
    # test model without alias
    NHITS(h=12, input_size=24, max_steps=1),
]
nf = NeuralForecast(models=models, freq='M')
nf.fit(df=AirPassengersPanel_train, static_df=AirPassengersStatic)
forecasts = nf.predict(futr_df=AirPassengersPanel_test)
test_eq(
    forecasts.columns.to_list(),
    ['unique_id', 'ds', 'AutoDIL', 'NHITSMQ-median', 'NHITSMQ-lo-80', 'NHITSMQ-hi-80', 'MyRNN', 'StemMulti', 'NHITS']
)

In [ ]:
#| hide
# Unit test for core/model interactions
config = {'input_size': tune.choice([12, 24]), 
          'hidden_size': 256,
          'max_steps': 1,
          'val_check_steps': 1,
          'step_size': 12}

config_drnn = {'input_size': tune.choice([-1]), 
               'encoder_hidden_size': tune.choice([5, 10]),
               'max_steps': 1,
               'val_check_steps': 1,
               'step_size': 1}

fcst = NeuralForecast(
    models=[
        AutoDilatedRNN(h=12, config=config_drnn, cpus=1, num_samples=2),
        DeepAR(h=12, input_size=24, max_steps=1,
               stat_exog_list=['airline1'], futr_exog_list=['trend']),
        DilatedRNN(h=12, input_size=-1, encoder_hidden_size=10, max_steps=1,
                   stat_exog_list=['airline1'],
                   futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
        RNN(h=12, input_size=-1, encoder_hidden_size=10, max_steps=1,
            inference_input_size=24,
            stat_exog_list=['airline1'],
            futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
        TCN(h=12, input_size=-1, encoder_hidden_size=10, max_steps=1,
            stat_exog_list=['airline1'],
            futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
        AutoMLP(h=12, config=config, cpus=1, num_samples=2),
        NBEATSx(h=12, input_size=12, max_steps=1,
                stat_exog_list=['airline1'],
                futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
        NHITS(h=12, input_size=24, loss=MQLoss(level=[80]), max_steps=1),
        NHITS(h=12, input_size=12, max_steps=1,
              stat_exog_list=['airline1'],
              futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
        MLP(h=12, input_size=12, max_steps=1,
            stat_exog_list=['airline1'],
            futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
        TFT(h=12, input_size=24, max_steps=1),
        VanillaTransformer(h=12, input_size=24, max_steps=1),
        Informer(h=12, input_size=24, max_steps=1),
        Autoformer(h=12, input_size=24, max_steps=1),
        FEDformer(h=12, input_size=24, max_steps=1),
        PatchTST(h=12, input_size=24, max_steps=1),
        TimesNet(h=12, input_size=24, max_steps=1),
        StemGNN(h=12, input_size=24, n_series=2, max_steps=1, scaler_type='robust'),
    ],
    freq='M'
)
fcst.fit(df=AirPassengersPanel_train, static_df=AirPassengersStatic)
forecasts = fcst.predict(futr_df=AirPassengersPanel_test)
forecasts

In [ ]:
#| hide
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
plot_df = pd.concat([AirPassengersPanel_train, forecasts.reset_index()]).set_index('ds')

plot_df[plot_df['unique_id']=='Airline1'].drop(['unique_id','trend','y_[lag12]'], axis=1).plot(ax=ax, linewidth=2)

ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()

In [ ]:
#| hide
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
plot_df = pd.concat([AirPassengersPanel_train, forecasts.reset_index()]).set_index('ds')

plot_df[plot_df['unique_id']=='Airline2'].drop(['unique_id','trend','y_[lag12]'], axis=1).plot(ax=ax, linewidth=2)

ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()

In [ ]:
#| hide
config = {'input_size': tune.choice([12, 24]), 
          'hidden_size': 256,
          'max_steps': 1,
          'val_check_steps': 1,
          'step_size': 12}

config_drnn = {'input_size': tune.choice([-1]), 
               'encoder_hidden_size': tune.choice([5, 10]),
               'max_steps': 1,
               'val_check_steps': 1,
               'step_size': 1}

fcst = NeuralForecast(
    models=[
        DilatedRNN(h=12, input_size=-1, encoder_hidden_size=10, max_steps=1),
        AutoMLP(h=12, config=config, cpus=1, num_samples=1),
        NHITS(h=12, input_size=12, max_steps=1)
    ],
    freq='M'
)
cv_df = fcst.cross_validation(df=AirPassengersPanel, static_df=AirPassengersStatic, n_windows=3, step_size=1)

In [ ]:
#| hide
#test cross validation no leakage
def test_cross_validation(df, static_df, h, test_size):
    if (test_size - h) % 1:
        raise Exception("`test_size - h` should be module `step_size`")
    
    n_windows = int((test_size - h) / 1) + 1
    Y_test_df = df.groupby('unique_id').tail(test_size)
    Y_train_df = df.drop(Y_test_df.index)
    config = {'input_size': tune.choice([12, 24]),
              'step_size': 12, 'hidden_size': 256, 'max_steps': 1, 'val_check_steps': 1}
    config_drnn = {'input_size': tune.choice([-1]), 'encoder_hidden_size': tune.choice([5, 10]),
                   'max_steps': 1, 'val_check_steps': 1}
    fcst = NeuralForecast(
        models=[
            AutoDilatedRNN(h=12, config=config_drnn, cpus=1, num_samples=1),
            DilatedRNN(h=12, input_size=-1, encoder_hidden_size=5, max_steps=1),
            RNN(h=12, input_size=-1, encoder_hidden_size=5, max_steps=1,
                stat_exog_list=['airline1'], futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
            TCN(h=12, input_size=-1, encoder_hidden_size=5, max_steps=1,
                stat_exog_list=['airline1'], futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
            AutoMLP(h=12, config=config, cpus=1, num_samples=1),
            MLP(h=12, input_size=12, max_steps=1, scaler_type='robust'),
            NBEATSx(h=12, input_size=12, max_steps=1,
                    stat_exog_list=['airline1'], futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
            NHITS(h=12, input_size=12, max_steps=1, scaler_type='robust'),
            NHITS(h=12, input_size=12, loss=MQLoss(level=[80]), max_steps=1),
            TFT(h=12, input_size=24, max_steps=1, scaler_type='robust'),
            VanillaTransformer(h=12, input_size=12, max_steps=1, scaler_type=None),
            Informer(h=12, input_size=12, max_steps=1, scaler_type=None),
            Autoformer(h=12, input_size=12, max_steps=1, scaler_type=None),
            FEDformer(h=12, input_size=12, max_steps=1, scaler_type=None),
            PatchTST(h=12, input_size=24, max_steps=1, scaler_type=None),
            TimesNet(h=12, input_size=24, max_steps=1, scaler_type='standard'),
            StemGNN(h=12, input_size=12, n_series=2, max_steps=1, scaler_type='robust'),
            DeepAR(h=12, input_size=24, max_steps=1,
               stat_exog_list=['airline1'], futr_exog_list=['trend']),
        ],
        freq='M'
    )
    fcst.fit(df=Y_train_df, static_df=static_df)
    Y_hat_df = fcst.predict(futr_df=Y_test_df)
    Y_hat_df = Y_hat_df.merge(Y_test_df, how='left', on=['unique_id', 'ds'])
    last_dates = Y_train_df.groupby('unique_id').tail(1)
    last_dates = last_dates[['unique_id', 'ds']].rename(columns={'ds': 'cutoff'})
    Y_hat_df = Y_hat_df.merge(last_dates, how='left', on='unique_id')
    
    #cross validation
    fcst = NeuralForecast(
        models=[
            AutoDilatedRNN(h=12, config=config_drnn, cpus=1, num_samples=1),
            DilatedRNN(h=12, input_size=-1, encoder_hidden_size=5, max_steps=1),
            RNN(h=12, input_size=-1, encoder_hidden_size=5, max_steps=1,
                stat_exog_list=['airline1'], futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
            TCN(h=12, input_size=-1, encoder_hidden_size=5, max_steps=1,
                stat_exog_list=['airline1'], futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
            AutoMLP(h=12, config=config, cpus=1, num_samples=1),
            MLP(h=12, input_size=12, max_steps=1, scaler_type='robust'),
            NBEATSx(h=12, input_size=12, max_steps=1,
                    stat_exog_list=['airline1'], futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
            NHITS(h=12, input_size=12, max_steps=1, scaler_type='robust'),
            NHITS(h=12, input_size=12, loss=MQLoss(level=[80]), max_steps=1),
            TFT(h=12, input_size=24, max_steps=1, scaler_type='robust'),
            VanillaTransformer(h=12, input_size=12, max_steps=1, scaler_type=None),
            Informer(h=12, input_size=12, max_steps=1, scaler_type=None),
            Autoformer(h=12, input_size=12, max_steps=1, scaler_type=None),
            FEDformer(h=12, input_size=12, max_steps=1, scaler_type=None),
            PatchTST(h=12, input_size=24, max_steps=1, scaler_type=None),
            TimesNet(h=12, input_size=24, max_steps=1, scaler_type='standard'),
            StemGNN(h=12, input_size=12, n_series=2, max_steps=1, scaler_type='robust'),
            DeepAR(h=12, input_size=24, max_steps=1,
               stat_exog_list=['airline1'], futr_exog_list=['trend']),
        ],
        freq='M'
    )
    Y_hat_df_cv = fcst.cross_validation(df, static_df=static_df, test_size=test_size, 
                                        n_windows=None)
    for col in ['ds', 'cutoff']:
        Y_hat_df_cv[col] = pd.to_datetime(Y_hat_df_cv[col].astype(str))
        Y_hat_df[col] = pd.to_datetime(Y_hat_df[col].astype(str))
    pd.testing.assert_frame_equal(
        Y_hat_df[Y_hat_df_cv.columns],
        Y_hat_df_cv,
        check_dtype=False,
    )

In [ ]:
#| hide
test_cross_validation(AirPassengersPanel, AirPassengersStatic, h=12, test_size=12)

In [ ]:
#| hide
# test save and load
config = {'input_size': tune.choice([12, 24]), 
          'hidden_size': 256,
          'max_steps': 1,
          'val_check_steps': 1,
          'step_size': 12}

config_drnn = {'input_size': tune.choice([-1]),
               'encoder_hidden_size': tune.choice([5, 10]),
               'max_steps': 1,
               'val_check_steps': 1}

fcst = NeuralForecast(
    models=[
        AutoRNN(h=12, config=config_drnn, cpus=1, num_samples=2, refit_with_val=True),
        DilatedRNN(h=12, input_size=-1, encoder_hidden_size=5, max_steps=1),
        AutoMLP(h=12, config=config, cpus=1, num_samples=2),
        NHITS(h=12, input_size=12, max_steps=1,
              futr_exog_list=['trend'], hist_exog_list=['y_[lag12]']),
        StemGNN(h=12, input_size=12, n_series=2, max_steps=1, scaler_type='robust')
    ],
    freq='M'
)
fcst.fit(AirPassengersPanel_train)
forecasts1 = fcst.predict(futr_df=AirPassengersPanel_test)
fcst.save(path='./examples/debug_run/', model_index=None, overwrite=True, save_dataset=True)

In [ ]:
#| hide
# test `enable_checkpointing=True` should generate chkpt
shutil.rmtree('lightning_logs')
fcst = NeuralForecast(
    models=[
        MLP(h=12, input_size=12, max_steps=10, val_check_steps=5, enable_checkpointing=True),
        RNN(h=12, input_size=-1, max_steps=10, val_check_steps=5, enable_checkpointing=True)
    ],
    freq='M'
)
fcst.fit(AirPassengersPanel_train)
last_log = f"lightning_logs/{os.listdir('lightning_logs')[-1]}"
no_chkpt_found = ~np.any([file.endswith('checkpoints') for file in os.listdir(last_log)])
test_eq(no_chkpt_found, False)

In [ ]:
#| hide
# test `enable_checkpointing=False` should not generate chkpt
shutil.rmtree('lightning_logs')
fcst = NeuralForecast(
    models=[
        MLP(h=12, input_size=12, max_steps=10, val_check_steps=5),
        RNN(h=12, input_size=-1, max_steps=10, val_check_steps=5)
    ],
    freq='M'
)
fcst.fit(AirPassengersPanel_train)
last_log = f"lightning_logs/{os.listdir('lightning_logs')[-1]}"
no_chkpt_found = ~np.any([file.endswith('checkpoints') for file in os.listdir(last_log)])
test_eq(no_chkpt_found, True)

In [ ]:
#| hide
fcst2 = NeuralForecast.load(path='./examples/debug_run/')
forecasts2 = fcst2.predict(futr_df=AirPassengersPanel_test)

In [ ]:
#| hide
pairwise_tuples = [('AutoRNN', 'RNN'), ('DilatedRNN','DilatedRNN'), ('AutoMLP','MLP'), ('NHITS','NHITS'), ('StemGNN','StemGNN')]
for model1, model2 in pairwise_tuples:
    np.allclose(forecasts1[model1], forecasts2[model2])

In [ ]:
#| hide
# test short time series
config = {'input_size': tune.choice([12, 24]), 
          'max_steps': 1,
          'val_check_steps': 1}

fcst = NeuralForecast(
    models=[
        AutoNBEATS(h=12, config=config, cpus=1, num_samples=2)],
    freq='M'
)

AirPassengersShort = AirPassengersPanel.tail(36+144).reset_index(drop=True)
forecasts = fcst.cross_validation(AirPassengersShort, val_size=48, n_windows=1)

In [ ]:
#| hide
# test validation scale BaseWindows

models = [NHITS(h=12, input_size=24, max_steps=50, scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
nf.fit(AirPassengersPanel_train,val_size=12)
valid_losses = nf.models[0].valid_trajectories
assert valid_losses[-1][1] < 40, 'Validation loss is too high'
assert valid_losses[-1][1] > 10, 'Validation loss is too low'

models = [NHITS(h=12, input_size=24, max_steps=50, scaler_type=None)]
nf = NeuralForecast(models=models, freq='M')
nf.fit(AirPassengersPanel_train,val_size=12)
valid_losses = nf.models[0].valid_trajectories
assert valid_losses[-1][1] < 40, 'Validation loss is too high'
assert valid_losses[-1][1] > 10, 'Validation loss is too low'

In [ ]:
#| hide
# test validation scale BaseRecurrent

nf = NeuralForecast(
    models=[LSTM(h=12,
                 input_size=-1,
                 loss=MAE(),
                 scaler_type='robust',
                 encoder_n_layers=2,
                 encoder_hidden_size=128,
                 context_size=10,
                 decoder_hidden_size=128,
                 decoder_layers=2,
                 max_steps=50,
                 val_check_steps=10,
                 )
    ],
    freq='M'
)
nf.fit(AirPassengersPanel_train,val_size=12)
valid_losses = nf.models[0].valid_trajectories
assert valid_losses[-1][1] < 100, 'Validation loss is too high'
assert valid_losses[-1][1] > 30, 'Validation loss is too low'

In [ ]:
#| hide
# Test order of variables does not affect validation loss

AirPassengersPanel_train['zeros'] = 0
AirPassengersPanel_train['large_number'] = 100000
AirPassengersPanel_train['available_mask'] = 1
AirPassengersPanel_train = AirPassengersPanel_train[['unique_id','ds','zeros','y','available_mask','large_number']]

models = [NHITS(h=12, input_size=24, max_steps=50, scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
nf.fit(AirPassengersPanel_train,val_size=12)
valid_losses = nf.models[0].valid_trajectories
assert valid_losses[-1][1] < 40, 'Validation loss is too high'
assert valid_losses[-1][1] > 10, 'Validation loss is too low'

models = [NHITS(h=12, input_size=24, max_steps=50, scaler_type=None)]
nf = NeuralForecast(models=models, freq='M')
nf.fit(AirPassengersPanel_train,val_size=12)
valid_losses = nf.models[0].valid_trajectories
assert valid_losses[-1][1] < 40, 'Validation loss is too high'
assert valid_losses[-1][1] > 10, 'Validation loss is too low'

In [ ]:
#| hide
# Test fit fails if variable not in dataframe

# Base Windows
models = [NHITS(h=12, input_size=24, max_steps=1, hist_exog_list=['not_included'], scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
test_fail(nf.fit,
          contains='historical exogenous variables not found in input dataset',
          args=(AirPassengersPanel_train,))

models = [NHITS(h=12, input_size=24, max_steps=1, futr_exog_list=['not_included'], scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
test_fail(nf.fit,
          contains='future exogenous variables not found in input dataset',
          args=(AirPassengersPanel_train,))

models = [NHITS(h=12, input_size=24, max_steps=1, stat_exog_list=['not_included'], scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
test_fail(nf.fit,
          contains='static exogenous variables not found in input dataset',
          args=(AirPassengersPanel_train,))

# Base Recurrent
models = [LSTM(h=12, input_size=24, max_steps=1, hist_exog_list=['not_included'], scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
test_fail(nf.fit,
          contains='historical exogenous variables not found in input dataset',
          args=(AirPassengersPanel_train,))

models = [LSTM(h=12, input_size=24, max_steps=1, futr_exog_list=['not_included'], scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
test_fail(nf.fit,
          contains='future exogenous variables not found in input dataset',
          args=(AirPassengersPanel_train,))

models = [LSTM(h=12, input_size=24, max_steps=1, stat_exog_list=['not_included'], scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
test_fail(nf.fit,
          contains='static exogenous variables not found in input dataset',
          args=(AirPassengersPanel_train,))

In [ ]:
#| hide
# Test passing unused variables in dataframe does not affect forecasts  

models = [NHITS(h=12, input_size=24, max_steps=5, hist_exog_list=['zeros'], scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
nf.fit(AirPassengersPanel_train)

Y_hat1 = nf.predict(df=AirPassengersPanel_train[['unique_id','ds','y','zeros','large_number']])
Y_hat2 = nf.predict(df=AirPassengersPanel_train[['unique_id','ds','y','zeros']])

pd.testing.assert_frame_equal(
    Y_hat1,
    Y_hat2,
    check_dtype=False,
)

models = [LSTM(h=12, input_size=24, max_steps=5, hist_exog_list=['zeros'], scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
nf.fit(AirPassengersPanel_train)

Y_hat1 = nf.predict(df=AirPassengersPanel_train[['unique_id','ds','y','zeros','large_number']])
Y_hat2 = nf.predict(df=AirPassengersPanel_train[['unique_id','ds','y','zeros']])

pd.testing.assert_frame_equal(
    Y_hat1,
    Y_hat2,
    check_dtype=False,
)

In [ ]:
#| hide
#| polars
import polars

In [ ]:
#| hide
#| polars
models = [LSTM(h=12, input_size=24, max_steps=5, hist_exog_list=['zeros'], scaler_type='robust')]
nf = NeuralForecast(models=models, freq='M')
nf.fit(AirPassengersPanel_train)
insample_preds = nf.predict_insample()
preds = nf.predict()
cv_res = nf.cross_validation(df=AirPassengersPanel_train)

renamer = {'unique_id': 'uid', 'ds': 'time', 'y': 'target'}
inverse_renamer = {v: k for k, v in renamer.items()}
AirPassengers_pl = polars.from_pandas(AirPassengersPanel_train)
AirPassengers_pl = AirPassengers_pl.rename(renamer)
nf = NeuralForecast(models=models, freq='1mo')
nf.fit(
    AirPassengers_pl, id_col='uid', time_col='time', target_col='target'
)
insample_preds_pl = nf.predict_insample()
preds_pl = nf.predict()
cv_res_pl = nf.cross_validation(
    df=AirPassengers_pl, id_col='uid', time_col='time', target_col='target'
)

def assert_equal_dfs(pandas_df, polars_df):
    mapping = {k: v for k, v in inverse_renamer.items() if k in polars_df}
    pd.testing.assert_frame_equal(
        pandas_df,
        polars_df.rename(mapping).to_pandas(),
    )

assert_equal_dfs(preds, preds_pl)
assert_equal_dfs(insample_preds, insample_preds_pl)
assert_equal_dfs(cv_res, cv_res_pl)